In [ ]:
import xlwings as xw

def fix_product_control(filepath):
    with xw.App(visible=False) as app:
        wb = app.books.open(filepath)
        ws = wb.sheets['Data']

        # ✅ Read headers from Row 2
        header = ws.range('A2').expand('right').value
        try:
            bf_col = header.index('Business Framework') + 1
            bfg_col = header.index('Business Framework Group') + 1
        except ValueError:
            raise ValueError("Column name not found!")

        # ✅ Find last row in first column
        last_row = ws.range('A' + str(ws.cells.last_cell.row)).end('up').row

        # ✅ Loop through data rows only, starting from row 3
        for row in range(3, last_row + 1):
            bf_val = ws.cells(row, bf_col).value
            if str(bf_val).strip().upper() == 'PRODUCT CONTROL':
                ws.cells(row, bfg_col).value = 'L3 - Financial Control & Tax'

        wb.save()
        wb.close()
        print("✅ PRODUCT CONTROL check done and saved.")

if __name__ == "__main__":
    fix_product_control("Base_Dashboard.xlsx")


In [ ]:
import xlwings as xw
import os
import shutil

def run_task_two():
    # === CONFIG ===
    master_file = "Base_Dashboard.xlsx"
    report_names = [
        "Investor Relations",
        "Stress Testing",
        "APEX (FC)",
        "Finance COO",
        "Financial Support"
    ]

    sheets_and_cells = [
        ("Index", "D3"),
        ("Summary", "D3"),
        ("P&L Item View", "B3"),
        ("DC by Country", "B3"),
        ("Headcount by Country", "B3"),
        ("Driller", "B3")
    ]

    with xw.App(visible=False) as app:
        for report_name in report_names:
            # 📌 Open a fresh copy each loop
            wb = app.books.open(master_file)
            
            # Get month from 'Dates'!C4
            month_text = wb.sheets['Dates'].range('C4').value  # e.g. 'May-25'
            month_part, yy_part = month_text.split('-')
            year_full = "20" + yy_part
            month_year = f"{month_part} {year_full}"  # 'May 2025'

            # Find Reporting sheet
            reporting_sheet = None
            for sh in wb.sheets:
                if 'Reporting' in sh.name:
                    reporting_sheet = sh
                    break

            if not reporting_sheet:
                raise ValueError("No sheet with 'Reporting' found.")

            # Update Reporting sheet name
            new_reporting_name = f"{month_year} Reporting"
            reporting_sheet.name = new_reporting_name

            # Update Reporting sheet cells
            reporting_ws = wb.sheets[new_reporting_name]
            reporting_ws.range("C20").value = report_name
            reporting_ws.range("B25").value = f"{month_year} Results"

            # Update other sheets/cells
            for sheet_name, cell in sheets_and_cells:
                ws = wb.sheets[sheet_name]
                ws.range(cell).value = report_name

            # Save
            output_name = f"Dashboard_{report_name}.xlsx"
            wb.save(os.path.join(os.getcwd(), output_name))
            wb.close()

        print("✅ All reports created!")

if __name__ == "__main__":
    run_task_two()


In [1]:
#Working fine for duplicating cuntries block with formula updation, but not handling blockof totals.- Task one

from openpyxl import load_workbook
from openpyxl.utils import get_column_letter, column_index_from_string
from openpyxl.worksheet.cell_range import CellRange
from openpyxl.formula.translate import Translator
import copy


def replicate_dashboard_block_with_translated_formulas(
    file_path,
    sheet_name,
    source_start_col,
    source_end_col,
    country_list,
    header_row=3,
    text_row=5
):
    wb = load_workbook(file_path)
    ws = wb[sheet_name]

    start_col_idx = column_index_from_string(source_start_col)
    end_col_idx = column_index_from_string(source_end_col)
    num_cols_to_copy = end_col_idx - start_col_idx + 1

    # Where to start inserting the first block (+2 means 1 blank col gap)
    current_col_idx = end_col_idx + 2

    for country in country_list:
        print(f"🔹 Adding block for: {country}")

        # --- Copy the block ---
        for row in ws.iter_rows(
            min_col=start_col_idx,
            max_col=end_col_idx,
            min_row=1,
            max_row=ws.max_row
        ):
            for i, source_cell in enumerate(row):
                new_col_idx = current_col_idx + i
                new_cell = ws.cell(row=source_cell.row, column=new_col_idx)

                # ✅ Copy value or translated formula
                if source_cell.data_type == 'f':
                    orig_formula = source_cell.value
                    origin = source_cell.coordinate
                    target = new_cell.coordinate
                    new_formula = Translator(orig_formula, origin=origin).translate_formula(target)
                    new_cell.value = new_formula
                else:
                    new_cell.value = source_cell.value

                # ✅ Copy style
                new_cell.font = copy.copy(source_cell.font)
                new_cell.border = copy.copy(source_cell.border)
                new_cell.fill = copy.copy(source_cell.fill)
                new_cell.number_format = copy.copy(source_cell.number_format)
                new_cell.protection = copy.copy(source_cell.protection)
                new_cell.alignment = copy.copy(source_cell.alignment)

        # --- Handle merged header row ---
        merged_start = get_column_letter(current_col_idx)
        merged_end = get_column_letter(current_col_idx + num_cols_to_copy - 1)
        new_merge_range = f"{merged_start}{header_row}:{merged_end}{header_row}"

        # Remove overlaps if needed
        for merge in list(ws.merged_cells.ranges):
            if CellRange(new_merge_range).coord in merge.coord:
                ws.merged_cells.ranges.remove(merge)

        ws.merge_cells(new_merge_range)
        ws[f"{merged_start}{header_row}"] = country  # ✅ Only top-left cell gets value

        # Copy header style
        source_header_cell = ws[f"{source_start_col}{header_row}"]
        new_header_cell = ws[f"{merged_start}{header_row}"]
        new_header_cell.font = copy.copy(source_header_cell.font)
        new_header_cell.border = copy.copy(source_header_cell.border)
        new_header_cell.fill = copy.copy(source_header_cell.fill)
        new_header_cell.number_format = copy.copy(source_header_cell.number_format)
        new_header_cell.protection = copy.copy(source_header_cell.protection)
        new_header_cell.alignment = copy.copy(source_header_cell.alignment)

        # --- Fill ROW 4 below merged header with same country name ---
        for i in range(num_cols_to_copy):
            col_letter = get_column_letter(current_col_idx + i)
            cell = ws[f"{col_letter}{header_row + 1}"]
            cell.value = country

            # Optional: match style from the same row in the source block
            source_cell = ws[f"{get_column_letter(start_col_idx + i)}{header_row + 1}"]
            cell.font = copy.copy(source_cell.font)
            cell.border = copy.copy(source_cell.border)
            cell.fill = copy.copy(source_cell.fill)
            cell.number_format = copy.copy(source_cell.number_format)
            cell.protection = copy.copy(source_cell.protection)
            cell.alignment = copy.copy(source_cell.alignment)

        # --- Handle merged text row ---
        text_merge_start = get_column_letter(current_col_idx)
        text_merge_end = get_column_letter(current_col_idx + (column_index_from_string("Y") - start_col_idx))
        new_text_merge_range = f"{text_merge_start}{text_row}:{text_merge_end}{text_row}"

        for merge in list(ws.merged_cells.ranges):
            if CellRange(new_text_merge_range).coord in merge.coord:
                ws.merged_cells.ranges.remove(merge)

        ws.merge_cells(new_text_merge_range)
        ws[f"{text_merge_start}{text_row}"] = ws[f"{source_start_col}{text_row}"].value

        text_source_cell = ws[f"{source_start_col}{text_row}"]
        text_new_cell = ws[f"{text_merge_start}{text_row}"]
        text_new_cell.font = copy.copy(text_source_cell.font)
        text_new_cell.border = copy.copy(text_source_cell.border)
        text_new_cell.fill = copy.copy(text_source_cell.fill)
        text_new_cell.number_format = copy.copy(text_source_cell.number_format)
        text_new_cell.protection = copy.copy(text_source_cell.protection)
        text_new_cell.alignment = copy.copy(text_source_cell.alignment)

        # Advance for next block (+1 for blank col)
        current_col_idx += num_cols_to_copy + 1

    wb.save(file_path)
    wb.close()
    print("✅✅ All blocks copied successfully with translated formulas, merged headers, and country name in row 4!")


# --- EXAMPLE ---
if __name__ == "__main__":
    replicate_dashboard_block_with_translated_formulas(
        file_path="your_dashboard.xlsx",
        sheet_name="Dashboard",
        source_start_col="V",
        source_end_col="AC",
        country_list=["Paris", "Hongkong", "India"],
        header_row=3,
        text_row=5
    )


Processing... /

KeyboardInterrupt: 

In [ ]:
import os
import pandas as pd

from openpyxl import load_workbook

# Set folder path where your Excel files are stored
folder_path = r'path_to_your_folder'  # <-- Replace this with your actual folder path

# To store flagged files and sheet names
issues = []

# Loop through each Excel file in the folder
for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file)
        try:
            wb = load_workbook(file_path, data_only=True)
            sheet_names = wb.sheetnames

            # Check only sheet 2 and 3 (index 1 and 2)
            for sheet_index in [1, 2]:
                if sheet_index < len(sheet_names):
                    ws = wb[sheet_names[sheet_index]]

                    for row in ws.iter_rows(min_row=1, values_only=True):
                        if row[2] == 'Check':  # Column C is index 2 (0-based)
                            # From Column E (index 4) onwards
                            values_to_check = row[4:]
                            if any(val != 0 and val is not None for val in values_to_check):
                                issues.append((file, sheet_names[sheet_index]))
                            break  # No need to search further in this sheet
        except Exception as e:
            print(f"Error processing {file}: {e}")

# Print the issues
if issues:
    print("❌ Files with failed validation:")
    for file_name, sheet_name in issues:
        print(f"File: {file_name}, Sheet: {sheet_name}")
else:
    print("✅ All files passed the validation checks.")


In [4]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox

# Required sheet and column mapping
required_sheets = {
    'RTN': 'RTN',
    'Location': 'Country',
    'GCB': 'GCB'
}

def select_workbook():
    """Prompts the user to select an Excel file."""
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    file_path = filedialog.askopenfilename(
        title="Select Excel Workbook",
        filetypes=[("Excel files", "*.xlsx")]
    )
    if not file_path:
        raise Exception("No file selected.")
    return file_path

def validate_workbook(file_path):
    """Validates presence of required sheets and columns."""
    try:
        xl = pd.ExcelFile(file_path)
    except Exception as e:
        raise Exception(f"Failed to open Excel file: {e}")

    for sheet, required_col in required_sheets.items():
        if sheet not in xl.sheet_names:
            raise Exception(f"Missing required sheet: '{sheet}'")

        df = xl.parse(sheet)
        df.columns = df.columns.str.strip()  # Remove extra spaces

        if required_col not in df.columns:
            raise Exception(f"Sheet '{sheet}' must contain column '{required_col}'")
        
        if df[required_col].isnull().any():
            raise Exception(f"Column '{required_col}' in sheet '{sheet}' contains blank values")

    print("Workbook validation successful.")

if __name__ == "__main__":
    try:
        file_path = select_workbook()
        validate_workbook(file_path)
        print(f"Processing file: {file_path}")
        # Continue with your processing logic here...
    except Exception as e:
        print(f"Error: {e}")
